In [5]:
import folium
import folium.plugins
from folium.plugins import MarkerCluster
import folium.plugins.antpath
from shapely import wkt, geometry
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import pandas as pd
import requests
import math
import json
from collections import defaultdict
import os

In [7]:
url_direction = "https://ds-route-service-api.pupuk.in/ors/v2/directions/driving-hgv"

headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': '',
        'Content-Type': 'application/json; charset=utf-8'
    }

# request body key
units = 'km'
preference = 'recommended'

dc_map = "DC5"
num_of_dc = "45"

dataset_data = f"desa_{num_of_dc}.xlsx"
distance_list_file = f'distance_list_{num_of_dc}.json'

# Output
routes_opt_file = f"routes_final_{num_of_dc}_all.json"
vehicle_route_opt = f'vehicle_route_opt_{dc_map}_{num_of_dc}_all.json'
vehicle_route_opt_all = f'vehicle_route_opt_{num_of_dc}_all.json'
file_map = f"map_routes_{dc_map}_{num_of_dc}.html"

In [9]:
# dataset consumer handled by dc banyumas
dataset = pd.read_excel(dataset_data,sheet_name='all_dc')
pd.set_option('display.max_columns', None)
dataset.head()

,Unnamed: 0,id,customers,address,kabkota,provinsi,alokasi_per_bulan,lat_customers,lon_customers,nearest_plant_existing,lat_plant_existing,lon_plant_existing,Urea,NPK,NPK_Kakao,Organik,Total,Kios,Desa,%kios/desa,realisasi_penjualan_kab,jumlah_transaksi_kab,over_under_stock_kab,realisasi_penjualan_desa_per_bulan,jumlah_transaksi_desa_per_bulan,urea_per_bulan,NPK_per_bulan,latitude,longitude,assigned_DC,DC_latitude,DC_longitude,estimasi_pengiriman,distance_to_DC_km,durations_in_min
0,3392,GPP PURWOREJO_KEDUNGMULYO,KEDUNGMULYO,-,KAB. CILACAP,JAWA TENGAH,4.44,-7.78104,109.848,GPP PURWOREJO,-7.72268,109.9468,40260.970,25706.808,15.405,3150.0,69133.183,146.0,269.0,0.542751,29700.305,1951.0,39432.878,11.041006,0.604399,12.472419,7.963695,-7.78104,109.848,DC1,-7.749836,109.96055,1,26.97995,28.233667
1,447,GPP PURWOREJO_BAYAN,BAYAN,-,KAB. PEMALANG,JAWA TENGAH,4.44,-7.71695,109.942,GPP PURWOREJO,-7.72268,109.9468,23799.170,17154.820,0.000,946.0,41899.990,137.0,212.0,0.646226,16031.500,1338.0,25868.490,7.562028,0.525943,9.355020,6.743247,-7.71695,109.942,DC1,-7.749836,109.96055,1,11.28071,12.790833
2,448,GPP PURWOREJO_BAYEM,BAYEM,-,KAB. TEMANGGUNG,JAWA TENGAH,4.44,-7.73209,109.894,GPP PURWOREJO,-7.72268,109.9468,12594.320,19278.109,16.748,1245.0,33134.177,65.0,266.0,0.244361,11881.355,1491.0,21252.822,4.466675,0.467105,3.945589,6.039508,-7.73209,109.894,DC1,-7.749836,109.96055,1,15.74003,16.546333
3,6963,GPP PURWOREJO_SRUWOH,SRUWOH,-,KAB. BANJARNEGARA,JAWA TENGAH,4.44,-7.81306,109.968,GPP PURWOREJO,-7.72268,109.9468,17069.411,17047.748,0.000,175.0,34292.159,72.0,266.0,0.270677,13060.500,1216.0,21231.659,4.909962,0.380952,5.347560,5.340773,-7.81306,109.968,DC1,-7.749836,109.96055,1,11.56325,17.684333
4,6964,GPP PURWOREJO_SRUWOHDUKUH,SRUWOHDUKUH,-,KAB. KENDAL,JAWA TENGAH,4.44,-7.75471,109.844,GPP PURWOREJO,-7.72268,109.9468,32671.130,24409.096,0.000,1150.0,58230.226,286.0,266.0,1.075188,18786.550,1095.0,39443.676,7.062613,0.343045,10.235316,7.646960,-7.75471,109.844,DC1,-7.749836,109.96055,1,23.07162,22.736833


In [10]:
data_by_dc = defaultdict(lambda: {'name': [],'coords': [], 'demands': [], 'address':[], 'provinsi':[]})

for _, row in dataset.iterrows():
    label = row['assigned_DC']
    lat = float(row['lat_customers'])
    lon = float(row['lon_customers'])
    name = str(row['customers'])
    demands = math.ceil(row['alokasi_per_bulan'])
    address = str(row['address'])
    provinsi = str(row['provinsi'])
    dc_lat = float(row['DC_latitude'])
    dc_lon = float(row['DC_longitude'])

    # Jika belum ada data untuk DC ini, tambahkan DC info ke indeks awal
    if not data_by_dc[label]['coords']:
        # Tambahkan koordinat DC di indeks awal
        data_by_dc[label]['name'].append(label)
        data_by_dc[label]['coords'].append([dc_lon, dc_lat])
        # Tambahkan demand awal untuk DC (0)
        data_by_dc[label]['demands'].append(0)
        # Tambahkan address dan provinsi awal untuk DC
        data_by_dc[label]['address'].append("-")
        data_by_dc[label]['provinsi'].append(provinsi)

    # Tambahkan koordinat centroid dan koordinat kios ke dalam list untuk label cluster yang sesuai
    data_by_dc[label]['name'].append(name)
    data_by_dc[label]['coords'].append([lon, lat])
    data_by_dc[label]['demands'].append(demands)
    data_by_dc[label]['address'].append(address)
    data_by_dc[label]['provinsi'].append(provinsi)

In [11]:
with open(distance_list_file, 'r') as f:
    distance_data = json.load(f)

distances_by_dc = {entry['DC']: entry['distances'] for entry in distance_data}

for dc, distance in data_by_dc.items():
    if dc in distances_by_dc:
        # Tambahkan jarak ke data_by_dc[dc]['distances']
        distance['distance_matrix'] = [[math.ceil(value) if value is not None else 0 for value in row] for row in distances_by_dc[dc]]
    else:
        # Jika DC tidak ditemukan, tambahkan jarak kosong
        distance['distance_matrix'] = []

In [12]:
""" Dipakai jika belum mendapatkan data jarak """
# headers = {
#         'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
#         'Authorization': '',
#         'Content-Type': 'application/json; charset=utf-8'
#     }

# url = f'http://34.34.220.116:8082/ors/v2/matrix/driving-hgv'


# for label, data in data_by_dc.items():
#     village_coords = data['coords']

#     request = {'locations': village_coords,
#             'profile': 'driving-hgv',
#             'metrics': ['distance'],
#             'units':'m'
#             }

#     try:
#         # Mengirim permintaan POST
#         response = requests.post(url=url, headers=headers, json=request)
#         response.raise_for_status()  # Raise error jika status code bukan 2xx

#         # Parsing hasil
#         village_matrix = response.json()
#         distances = village_matrix.get('distances', [])

#         processed_matrix = [[math.ceil(value) if value is not None else 0 for value in row] for row in distances]

#         # Simpan hasil ke data_by_dc
#         data_by_dc[label]['distance_matrix'] = processed_matrix

#         print("{} Calculated {}x{} routes.".format(label,len(distances), len(distances[0])))

#     except requests.exceptions.RequestException as e:
#         response = response.json()
#         print("An error occurred while making the request:")
#         print(f"Code: {response['error']['code']}\nMessage: {response['error']['message']}")

#     except KeyError as e:
#         print("Response format error. Missing key:", e)

#     except Exception as e:
#         print("An unexpected error occurred:", e)

' Dipakai jika belum mendapatkan data jarak '

In [13]:
def split_demands(data):
    max_capacity = data["vehicle_capacities"][0]  # Asumsi semua kendaraan memiliki kapasitas yang sama
    new_name = []
    new_address = []
    new_province = []

    new_demands = []
    new_distance_matrix = []
    new_locations = []
    mapping = []  # Untuk melacak indeks baru ke indeks asli dan sub bagian

    for i, demand in enumerate(data["demands"]):
        if demand > max_capacity:
            # Hitung jumlah split yang diperlukan
            num_splits = -(-demand // max_capacity)  # Ceiling division
            num_splits = int(num_splits)
            split_values = [max_capacity] * (num_splits - 1) + [demand % max_capacity or max_capacity]

            # Tambahkan demand hasil split
            for part, split_value in enumerate(split_values):
                new_demands.append(split_value)
                mapping.append((i, part + 1))  # (konsumen asli, bagian ke-n)

            # Duplikasi baris dan kolom di distance_matrix untuk konsumen ini
            for _ in split_values:
                new_distance_matrix.append(data["distance_matrix"][i])

            for _ in split_values:
                new_locations.append(data['locations'][i])

            for _ in split_values:
                new_name.append(data['name'][i])

        else:
            new_demands.append(demand)
            mapping.append((i, 0))  # (konsumen asli, tidak di-split)

            new_name.append(data['name'][i])
            new_address.append(data['address'][i])
            new_province.append(data['province'][i])

            new_distance_matrix.append(data["distance_matrix"][i])
            new_locations.append(data['locations'][i])

    # Perbarui distance_matrix dengan menambahkan kolom baru
    updated_matrix = []
    for i, row in enumerate(new_distance_matrix):
        updated_row = []
        for j, _ in enumerate(new_distance_matrix):
            if i < len(mapping) and j < len(mapping):
                updated_row.append(data["distance_matrix"][mapping[i][0]][mapping[j][0]])  # Ambil jarak dari indeks asli
            else:
                updated_row.append(0)  # Atur jarak untuk posisi baru yang di-split
        updated_matrix.append(updated_row)


    return {
        "name": new_name,
        "address": new_address,
        "province": new_province,
        "distance_matrix": updated_matrix,
        "demands": new_demands,
        'locations': new_locations,
        "vehicle_capacities": data["vehicle_capacities"],
        "num_vehicles": data["num_vehicles"],
        "depot": data["depot"],
        "mapping": mapping,
    }


In [14]:
# data insert
# (When there's only one vehicle, it reduces to the Traveling Salesperson Problem.)
# A better way to define optimal routes is to minimize the length of the longest single route among all vehicles. This is the right definition if the goal is to complete all deliveries as soon as possible.
# djikstra, mencari minimum weight dari satu titik ke titik lain. TSP = djikstra + rute kembali. VRP = TSP dengan vehicle > 1

num_vehicle = 2000
vehicle_capacity = 20000 # satuannya bisa apapun
def create_data_model():
    all_data = {}
    """Stores the data for the problem."""
    for dc, dc_data in data_by_dc.items():
        data = {}
        data['name'] = dc_data['name']
        data['address'] = dc_data['address']
        data['province'] = dc_data['provinsi']
        data["locations"] = dc_data['coords']
        data['distance_matrix'] = dc_data['distance_matrix']
        data["demands"] = dc_data['demands']
        data["vehicle_capacities"] = [vehicle_capacity for _ in range(num_vehicle)]
        data["num_vehicles"] = num_vehicle
        data["depot"] = 0

        # Masukkan data ke dictionary hasil dengan key sesuai DC
        all_data[dc] = split_demands(data)

    return all_data

In [15]:
"""Capacited Vehicles Routing Problem (CVRP) using OR-TOOLS."""

result_data = []
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    mapping = data["mapping"]
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    total_trips = 0
    vehicle_id = 0

    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        remaining_load_capacity = data["vehicle_capacities"][vehicle_id]  # Track the remaining capacity

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            consumer_original, split_part = mapping[node_index]  # Ambil indeks asli dan bagian split
            consumer_label = f"{consumer_original}-{split_part}"

            route_load += data["demands"][node_index]
            remaining_load_capacity -= data["demands"][node_index]  # Subtract the demand from remaining capacity

            if node_index == data['depot']:
                plan_output += f"Depot: {data['name'][node_index]} | Cumm_Load: {route_load} | rem_capacity: {remaining_load_capacity} -> "
            elif data['demands'][node_index] == 0:
                index = solution.Value(routing.NextVar(index))
                continue
            else:
                plan_output += f"Consumer: {data['name'][node_index]} | Demand: {data['demands'][node_index]} | Cumm_Load: {route_load} | rem_capacity: {remaining_load_capacity} -> "

            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )

            # Ambil data lokasi pertama dan kedua
            lat_current, lon_current = data['locations'][node_index]  # Koordinat untuk node saat ini
            # lat_next, lon_next = data['locations'][index]  # Koordinat untuk node berikutnya

            # Append data for DataFrame
            # Append current and next node to result_data
            if route_load != 0:
                result_data.append({
                    "Vehicle": vehicle_id,
                    "Depot": data['depot'],
                    "Node": consumer_label,
                    "Demand": data["demands"][node_index],
                    "Cumulative Load": route_load,
                    "Remaining Capacity": remaining_load_capacity,
                    "Distance": route_distance,
                    "Current Node": node_index,
                    "Current_lat": lat_current,
                    "Current_lon": lon_current,
                    "Next Node": index,
                })
        # After visiting all customers, return to depot (node 0)
        node_index = manager.IndexToNode(previous_index)
        consumer_original, split_part = mapping[node_index]
        # route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

        if route_load != 0:
            plan_output += f"\nLast Customer: {data['name'][node_index]} Load({route_load})\n"
            plan_output += f"Distance of the route: {route_distance}m\n"
            plan_output += f"Load of the route: {route_load}\n"
            total_trips += 1
            total_distance += route_distance
            total_load += route_load
            print(plan_output)

    print(f"Total number of trips: {total_trips}")
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")

def get_routes(solution, routing, manager, dc, data, json_file='routes.json'):
    total_distance = 0
    total_load = 0
    total_trips = 0

    routes = {}

    if dc not in routes:
        routes[dc] = {"summary": None, "routes": {}}

    if "routes" not in routes[dc]:
        routes[dc]["routes"] = {}

    for vehicle_id in range(routing.vehicles()):
        index = routing.Start(vehicle_id)
        route_distance = 0
        route_load = 0
        remaining_load_capacity = data["vehicle_capacities"][vehicle_id]
        route = []

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            remaining_load_capacity -= data["demands"][node_index]
            if data['demands'][node_index] != 0 or node_index == data['depot']:
                data_route = {
                    'name': data['name'][node_index],
                    'location': data['locations'][node_index],
                    'remaining_load_capacity': remaining_load_capacity,
                    'route_load': route_load,
                    'demands': data['demands'][node_index],
                }
                route.append({
                    'routing': data_route
                })

                previous_index = index
                index = solution.Value(routing.NextVar(index))
                route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
                )

            else:
                index = solution.Value(routing.NextVar(index))
                continue
        node_index = manager.IndexToNode(index)
        end_route = {
            'name': data['name'][node_index],
            'location': data['locations'][node_index],
            'remaining_load_capacity': remaining_load_capacity,
            'route_load': route_load,
            'demands': data['demands'][node_index],
        }
        route.append({
            'routing': end_route
        })

        if route_load != 0:
            total_distance += route_distance
            total_load += route_load
            total_trips += 1

        # Simpan ke dalam dictionary hanya jika rute memiliki lebih dari dua lokasi (depot + customer)

        if len(route) != 2:
            routes[dc]["routes"][vehicle_id] = route
    # Tambahkan node terakhir (end/depot)
    node_index = manager.IndexToNode(index)
    summary = {
        'distance': total_distance,
        'load':total_load,
        'trips': total_trips
    }

    routes[dc]["summary"] = summary


    update_routes_json(dc, routes, json_file)

def update_routes_json(dc, routes, json_file='routes.json'):
    # Cek apakah file JSON sudah ada
    if os.path.exists(json_file):
        # Jika ada, baca data yang ada dalam file
        with open(json_file, 'r') as f:
            data = json.load(f)
    else:
        # Jika tidak ada, buat file kosong dengan struktur dasar
        data = {}

    # Jika dc belum ada dalam data, buatkan list baru untuk dc
    if dc not in data:
        data[dc] = {}
        data[dc].update(routes[dc])
            # Simpan kembali ke dalam file JSON
        with open(json_file, 'w') as f:
            json.dump(data, f, indent=4)

        print(f"Routes data for {dc} successfuly inserted in {json_file}")
    else:
        print(f"{dc} optimization already in data!")


def main():
    """Solve the CVRP problem."""

    data = create_data_model()

    for dc, dc_data in data.items():
        # Create the routing index manager.
        manager = pywrapcp.RoutingIndexManager(
            len(dc_data["distance_matrix"]), dc_data["num_vehicles"], dc_data["depot"]
        )

        # Create Routing Model.
        routing = pywrapcp.RoutingModel(manager)

        # distance_matrix = compute_haversine_distance_matrix(dc_data['locations'])
        # Create and register a transit callback (menghitung jarak antar dua titik).
        def distance_callback(from_index, to_index):
            """Returns the distance between the two nodes."""
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return dc_data["distance_matrix"][from_node][to_node]

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)


        # Define cost of each arc.
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # Add Capacity constraint.
        def demand_callback(from_index):
            """Returns the demand of the node."""
            # Convert from routing variable Index to demands NodeIndex.
            from_node = manager.IndexToNode(from_index)
            return dc_data["demands"][from_node]

        demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
        routing.AddDimensionWithVehicleCapacity(
            demand_callback_index,
            0,  # null capacity slack
            dc_data["vehicle_capacities"],  # vehicle maximum capacities
            True,  # start cumul to zero
            "Capacity",
        )

        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()

        # menggunakan pendekatan pencarian Guided_local_search untuk menghindari pengambilan local optimum
        # search_parameters.first_solution_strategy = (
        #     routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
        # )

        search_parameters.local_search_metaheuristic = (
            routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
        )
        search_parameters.time_limit.FromSeconds(15)
        # search_parameters.log_search = True

        if os.path.exists(routes_opt_file):
            # Jika ada, baca data yang ada dalam file
            with open(routes_opt_file, 'r') as f:
                data_json = json.load(f)
        else:
            # Jika tidak ada, buat file kosong dengan struktur dasar
            data_json = {}

        # Jika dc belum ada dalam data_json, buatkan list baru untuk dc
        if dc not in data_json:
            print(f"Calculating {dc}...")
            solution = routing.SolveWithParameters(search_parameters)

            # # Print solution on console.
            # if solution:
            #     print_solution(dc_data, manager, routing, solution)
            # else:
            #     print('No Solution Found!')

            if not solution:
                print(f'{dc} No Solution Found!')
                # continue
            get_routes(solution, routing, manager, dc, dc_data, routes_opt_file)
        else:
            print(f"{dc} optimization already in data!")
            # continue


        print(f"{dc} end of program!")

if __name__ == "__main__":
    main()

DC1 optimization already in data!
DC1 end of program!
DC2 optimization already in data!
DC2 end of program!
DC3 optimization already in data!
DC3 end of program!
DC4 optimization already in data!
DC4 end of program!
DC5 optimization already in data!
DC5 end of program!
DC6 optimization already in data!
DC6 end of program!
DC7 optimization already in data!
DC7 end of program!
DC8 optimization already in data!
DC8 end of program!
DC9 optimization already in data!
DC9 end of program!
DC10 optimization already in data!
DC10 end of program!
DC11 optimization already in data!
DC11 end of program!
DC12 optimization already in data!
DC12 end of program!
DC13 optimization already in data!
DC13 end of program!
DC14 optimization already in data!
DC14 end of program!
DC15 optimization already in data!
DC15 end of program!
DC16 optimization already in data!
DC16 end of program!
DC17 optimization already in data!
DC17 end of program!
DC18 optimization already in data!
DC18 end of program!
DC19 optim

In [16]:
with open(routes_opt_file, "r") as json_file:
    routes = json.load(json_file)

coordinates_dict = {}

for dc, dc_data in routes.items():
    coordinates_dict[dc] = {}
    route = dc_data['routes']
    for vehicle_id, path in route.items():
        coordinates_list = [(stop['routing']['location'][0], stop['routing']['location'][1]) for stop in path]
        coordinates_dict[dc][str(vehicle_id)] = coordinates_list

In [17]:
total_trips = 0
total_distance = 0
total_load = 0
for dc, vehicle_data in routes.items():  # Iterasi level pertama (key: DC45, value: dictionary)
    distance = vehicle_data['summary']['distance']
    load = vehicle_data['summary']['load']
    trips = vehicle_data['summary']['trips']
    print(f"{dc}, load: {load}, Trips: {trips}")
    total_trips += trips
    total_distance += distance
    total_load += load

print("\nCalculated based on OR-TOOLS")
print(f"Total trips: {total_trips}")
print(f"Total distance: {total_distance} meter")
print(f"Total Load: {total_load} Ton")


DC1, load: 1835, Trips: 1
DC2, load: 4213, Trips: 1
DC3, load: 1852, Trips: 1
DC4, load: 3842, Trips: 1
DC5, load: 2850, Trips: 1
DC6, load: 2709, Trips: 1
DC7, load: 3240, Trips: 1
DC8, load: 1722, Trips: 1
DC9, load: 3675, Trips: 1
DC10, load: 4017, Trips: 1
DC11, load: 4122, Trips: 1
DC12, load: 3804, Trips: 2
DC13, load: 5068, Trips: 1
DC14, load: 2545, Trips: 1
DC15, load: 2076, Trips: 1
DC16, load: 3399, Trips: 1
DC17, load: 2207, Trips: 1
DC18, load: 5933, Trips: 1
DC19, load: 5846, Trips: 1
DC20, load: 2290, Trips: 1
DC21, load: 3920, Trips: 1
DC22, load: 3802, Trips: 1
DC23, load: 1542, Trips: 1
DC24, load: 2172, Trips: 1
DC25, load: 3195, Trips: 1
DC26, load: 2853, Trips: 1
DC27, load: 2156, Trips: 1
DC28, load: 1844, Trips: 1
DC29, load: 3787, Trips: 1
DC30, load: 5991, Trips: 1
DC31, load: 1931, Trips: 2
DC32, load: 2185, Trips: 1
DC33, load: 2577, Trips: 1
DC34, load: 2242, Trips: 2
DC35, load: 2373, Trips: 1
DC36, load: 2436, Trips: 1
DC37, load: 3538, Trips: 1
DC38, load

In [18]:
def decode_polyline(polyline, is3d=False):
    """Decodes a Polyline string into a GeoJSON geometry.
    :param polyline: An encoded polyline, only the geometry.
    :type polyline: string
    :param is3d: Specifies if geometry contains Z component.
    :type is3d: boolean
    :returns: GeoJSON Linestring geometry
    :rtype: dict
    """
    points = []
    index = lat = lng = z = 0

    while index < len(polyline):
        result = 1
        shift = 0
        while True:
            b = ord(polyline[index]) - 63 - 1
            index += 1
            result += b << shift
            shift += 5
            if b < 0x1F:
                break
        lat += (~result >> 1) if (result & 1) != 0 else (result >> 1)

        result = 1
        shift = 0
        while True:
            b = ord(polyline[index]) - 63 - 1
            index += 1
            result += b << shift
            shift += 5
            if b < 0x1F:
                break
        lng += ~(result >> 1) if (result & 1) != 0 else (result >> 1)

        if is3d:
            result = 1
            shift = 0
            while True:
                b = ord(polyline[index]) - 63 - 1
                index += 1
                result += b << shift
                shift += 5
                if b < 0x1F:
                    break
            if (result & 1) != 0:
                z += ~(result >> 1)
            else:
                z += result >> 1

            points.append(
                [
                    round(lng * 1e-5, 6),
                    round(lat * 1e-5, 6),
                    round(z * 1e-2, 1),
                ]
            )

        else:
            points.append([round(lng * 1e-5, 6), round(lat * 1e-5, 6)])

    geojson = {u"type": u"LineString", u"coordinates": points}

    return geojson

## Menggambarkan rute per DC

In [19]:
data = create_data_model()
data_dc = dataset[dataset['assigned_DC'] == dc_map]
data_dc

,Unnamed: 0,id,customers,address,kabkota,provinsi,alokasi_per_bulan,lat_customers,lon_customers,nearest_plant_existing,lat_plant_existing,lon_plant_existing,Urea,NPK,NPK_Kakao,Organik,Total,Kios,Desa,%kios/desa,realisasi_penjualan_kab,jumlah_transaksi_kab,over_under_stock_kab,realisasi_penjualan_desa_per_bulan,jumlah_transaksi_desa_per_bulan,urea_per_bulan,NPK_per_bulan,latitude,longitude,assigned_DC,DC_latitude,DC_longitude,estimasi_pengiriman,distance_to_DC_km,durations_in_min
684,8194,GD BGR CANDI SEMARANG_WONOSARI,WONOSARI,-,KAB. PEMALANG,JAWA TENGAH,1.22,-6.97933,110.309,GD BGR CANDI SEMARANG,-7.01446,110.3477,23799.170,17154.820,0.000,946.0,41899.990,137.0,212.0,0.646226,16031.500,1338.0,25868.490,7.562028,0.525943,9.355020,6.743247,-6.97933,110.309,DC5,-6.961605,110.159086,1,24.93482,24.233667
685,8218,BGR JENARSARI_WONOTENGGANG,WONOTENGGANG,-,KAB. BLORA,JAWA TENGAH,16.97,-6.95100,110.087,BGR JENARSARI,-6.95329,110.1013,65815.932,50754.146,0.000,40000.0,156570.078,366.0,271.0,1.350554,58990.675,2259.0,97579.403,21.767777,0.694649,20.238601,15.607056,-6.95100,110.087,DC5,-6.961605,110.159086,3,16.19117,16.235167
686,8164,GD BGR CANDI SEMARANG_WONOPLUMBON,WONOPLUMBON,-,KAB. BLORA,JAWA TENGAH,1.22,-7.03920,110.285,GD BGR CANDI SEMARANG,-7.01446,110.3477,65815.932,50754.146,0.000,40000.0,156570.078,366.0,271.0,1.350554,58990.675,2259.0,97579.403,21.767777,0.694649,20.238601,15.607056,-7.03920,110.285,DC5,-6.961605,110.159086,1,33.45635,36.947667
687,3665,Kendal - Wonorejo_KERTOMULYO,KERTOMULYO,-,KAB. KLATEN,JAWA TENGAH,16.97,-6.98042,110.203,Kendal - Wonorejo,-6.94013,110.2557,23536.540,16313.995,0.000,2906.0,42756.535,156.0,391.0,0.398977,10453.550,782.0,32302.985,2.673542,0.166667,5.016313,3.476981,-6.98042,110.203,DC5,-6.961605,110.159086,3,11.25900,14.985667
688,4823,Kendal - Wonorejo_NGAMPEL WETAN,NGAMPEL WETAN,-,KAB. KENDAL,JAWA TENGAH,16.97,-6.97953,110.194,Kendal - Wonorejo,-6.94013,110.2557,32671.130,24409.096,0.000,1150.0,58230.226,286.0,266.0,1.075188,18786.550,1095.0,39443.676,7.062613,0.343045,10.235316,7.646960,-6.97953,110.194,DC5,-6.961605,110.159086,3,9.26362,11.656667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868,1787,BGR JENARSARI_GONDANG,GONDANG,-,KAB. BANJARNEGARA,JAWA TENGAH,16.97,-6.93350,110.138,BGR JENARSARI,-6.95329,110.1013,17069.411,17047.748,0.000,175.0,34292.159,72.0,266.0,0.270677,13060.500,1216.0,21231.659,4.909962,0.380952,5.347560,5.340773,-6.93350,110.138,DC5,-6.961605,110.159086,3,9.91705,11.655500
869,6489,BGR JENARSARI_SENDANGDAWUNG,SENDANGDAWUNG,-,KAB. CILACAP,JAWA TENGAH,16.97,-6.92006,110.096,BGR JENARSARI,-6.95329,110.1013,40260.970,25706.808,15.405,3150.0,69133.183,146.0,269.0,0.542751,29700.305,1951.0,39432.878,11.041006,0.604399,12.472419,7.963695,-6.92006,110.096,DC5,-6.961605,110.159086,3,18.97578,24.106000
870,6488,BGR JENARSARI_SENDANGDAWUHAN,SENDANGDAWUHAN,-,KAB. BANJARNEGARA,JAWA TENGAH,16.97,-6.93454,110.074,BGR JENARSARI,-6.95329,110.1013,17069.411,17047.748,0.000,175.0,34292.159,72.0,266.0,0.270677,13060.500,1216.0,21231.659,4.909962,0.380952,5.347560,5.340773,-6.93454,110.074,DC5,-6.961605,110.159086,3,19.13504,20.152500
871,8161,GD BGR CANDI SEMARANG_WONOLOPO,WONOLOPO,-,KAB. WONOGIRI,JAWA TENGAH,1.22,-7.05165,110.306,GD BGR CANDI SEMARANG,-7.01446,110.3477,35313.661,33175.433,89.095,4466.0,73044.189,168.0,251.0,0.669323,18539.150,1514.0,54505.039,7.386116,0.502656,11.724323,11.014420,-7.05165,110.306,DC5,-6.961605,110.159086,1,34.47528,38.592333


In [20]:
# See what a 'random' tour would have been
coords = data[dc_map]['locations']
coords.append(coords[0])


request = {'coordinates': coords,
           'geometry': 'true',
           'radiuses':-1,
           'preference': preference,
           'units':units
           }

random_route = requests.post(url=url_direction,headers=headers,json=request)
random_route = random_route.json()
random_route = [random_route]

In [22]:
vehicle_id_list = []
for dc, inner_dict in coordinates_dict.items():
    for year in inner_dict.keys():
        vehicle_id_list.append(year)

In [23]:
# Memuat data result_route_optimal_all jika file sudah ada
if os.path.exists(vehicle_route_opt_all):
    with open(vehicle_route_opt_all, 'r') as f:
        result_route_optimal_all = json.load(f)
        print(f"{vehicle_route_opt_all} already exists!")
else:
    print(f"Creating: {vehicle_route_opt_all}")
    result_route_optimal_all = {}
    for dc, dc_data in coordinates_dict.items():
        checking_vehicle_id = dc_data.keys()
        vehicle_routes = {}  # Menyimpan hasil rute untuk vehicle_id ini

        for vehicle_id_inner, coords in dc_data.items():
              coordinates = coords
              print(f"Creating data JSON for {vehicle_id_inner} in {dc}")
              request['coordinates'] = coordinates

              # Mengirimkan POST request untuk mendapatkan optimal route
              optimal_route_response = requests.post(url=url_direction, headers=headers, json=request)

              if optimal_route_response.status_code == 200:
                  optimal_route = optimal_route_response.json()
                  vehicle_routes[vehicle_id_inner] = optimal_route
              else:
                  json_response = optimal_route_response.json()
                  print(f"Error with vehicle {vehicle_id_inner} in {dc}: {json_response}")
                  vehicle_routes[vehicle_id_inner] = {'error': f"Error {json_response}"}

        result_route_optimal_all[dc] = vehicle_routes

        # Simpan hasil ke file JSON setiap selesai memproses satu vehicle_id
        with open(vehicle_route_opt_all, 'w') as file_json:
            json.dump(result_route_optimal_all, file_json, indent=4)

    print("End of calculation")

vehicle_route_opt_45_all.json already exists!


In [24]:
total_distance = 0
total_duration = 0
for dc, vehicles in result_route_optimal_all.items():
    for vehicle_id, details in vehicles.items():
        routes = details.get('routes', [])
        for route in routes:
            summary = route.get('summary', {})
            distance = summary.get('distance', 0)
            duration = summary.get('duration', 0)

            total_distance += distance
            total_duration += duration

            print(f"DC: {dc}, Vehicle ID: {vehicle_id}, Distance: {distance}, Duration: {duration}")

print(f"\nTotal Distance: {total_distance}")
print(f"Total Duration: {total_duration}")

DC: DC1, Vehicle ID: 1999, Distance: 874.838, Duration: 118011.90000000007
DC: DC2, Vehicle ID: 1999, Distance: 443.431, Duration: 51709.40000000001
DC: DC4, Vehicle ID: 1999, Distance: 387.476, Duration: 41779.79999999998
DC: DC5, Vehicle ID: 1999, Distance: 667.395, Duration: 87387.49999999997
DC: DC6, Vehicle ID: 1999, Distance: 635.432, Duration: 70519.4
DC: DC7, Vehicle ID: 1999, Distance: 371.703, Duration: 39552.49999999999
DC: DC8, Vehicle ID: 1999, Distance: 552.394, Duration: 63189.4
DC: DC9, Vehicle ID: 1999, Distance: 618.76, Duration: 75429.69999999997
DC: DC11, Vehicle ID: 0, Distance: 381.763, Duration: 44151.3
DC: DC12, Vehicle ID: 0, Distance: 2.234, Duration: 160.8
DC: DC12, Vehicle ID: 1999, Distance: 615.695, Duration: 82868.10000000003
DC: DC13, Vehicle ID: 1999, Distance: 331.85, Duration: 34037.799999999996
DC: DC14, Vehicle ID: 1999, Distance: 814.307, Duration: 92210.4
DC: DC15, Vehicle ID: 1999, Distance: 782.227, Duration: 92493.00000000009
DC: DC16, Vehicle 

## Pemetaan berdasarkan DC

In [29]:
# Daftar vehicle_id yang ingin diproses
vehicle_id_list = [vehicle_id for vehicle_id, coords in coordinates_dict[dc_map].items()]

if os.path.exists(vehicle_route_opt):
    # Jika ada, baca data yang ada dalam file
    with open(vehicle_route_opt, 'r') as f:
        result_route_optimal = json.load(f)
else:
    # Jika tidak ada, buat file kosong dengan struktur dasar
    result_route_optimal = []

# Mengirimkan request untuk setiap vehicle_id
for vehicle_id in vehicle_id_list:
    found = False  # Flag untuk mengecek apakah vehicle_id ditemukan
    for value in result_route_optimal:
        if vehicle_id in value:  # Mengecek apakah vehicle_id adalah kunci dari dictionary
            found = True
            print(f"{vehicle_id} already exist")
    if not found:
        print(f"{vehicle_id} Not found in {vehicle_route_opt}")
        print(f"Creating data JSON for {vehicle_id}")
        if vehicle_id in coordinates_dict[dc_map]:
            coordinates = coordinates_dict[dc_map][vehicle_id]
            request['coordinates'] = coordinates

            # Mengirimkan POST request untuk mendapatkan optimal route
            optimal_route_response = requests.post(url=url_direction, headers=headers, json=request)

            # Mengonversi response ke format JSON
            if optimal_route_response.status_code == 200:
                optimal_route = optimal_route_response.json()
                # Menambahkan hasil response ke dalam list result_route_optimal
                result_route_optimal.append({
                    f'{vehicle_id}': optimal_route
                })
            else:
                print(f"Error with vehicle {vehicle_id}: {optimal_route_response.status_code}")
                # Menambahkan error response ke dalam list
                result_route_optimal.append({
                    'vehicle_id': vehicle_id,
                    'error': f"Error {optimal_route_response.status_code}"
                })
        else:
            print(f"Coordinates not found for vehicle {vehicle_id}")
            # Menambahkan error response ke dalam list jika koordinat tidak ditemukan
            result_route_optimal.append({
                'vehicle_id': vehicle_id,
                'error': 'Coordinates not found'
            })
        with open(vehicle_route_opt,'w') as file_json:
            json.dump(result_route_optimal, file_json, indent=4)


print("end of calculation")

1999 Not found in vehicle_route_opt_DC5_45_all.json
Creating data JSON for 1999
end of calculation


In [30]:
distance_list = []
duration_list = []
for i, value in enumerate(result_route_optimal):
    for vehicle_id in vehicle_id_list:

        if vehicle_id in value:
            routes = value[vehicle_id]['routes']
            coords = value[vehicle_id]['metadata']['query']['coordinates']

            for k, route in enumerate(routes):
                geometry = route['geometry']
                distance = route['summary']['distance']
                duration = route['summary']['duration']
                distance_list.append(distance)
                duration_list.append(duration)

sum_distance = sum(distance_list)
sum_duration = sum(duration_list) / 60
mean_distance = sum(distance_list) / len(distance_list)
mean_duration = (sum(duration_list) / 60) / len(duration_list)

print(f"Sum of Distance {preference}: {sum_distance:.0f} {units}")
print(f"Sum of Duration {preference}: {math.ceil(sum_duration):.0f} hours")

print(f"\nAverage of Distance {preference}: {mean_distance:.0f} {units}")
print(f"Average of Duration {preference}: {mean_duration:.0f} hours")

print(f"\nNumber of Amount Trips before optimization in {dc_map} {(sum(data_dc['alokasi_per_bulan']) + 183) / 8:.0f}")
print(f"Number of Amount Trips after optimization in {dc_map}: {len(vehicle_id_list)}")

Sum of Distance recommended: 667 km
Sum of Duration recommended: 1457 hours

Average of Distance recommended: 667 km
Average of Duration recommended: 1456 hours

Number of Amount Trips before optimization in DC5 376
Number of Amount Trips after optimization in DC5: 1


In [31]:
depot_loc = data_dc[['DC_longitude','DC_latitude']].head(1).values.tolist()
consumer_loc = data_dc[['lon_customers', 'lat_customers']].iloc[1:].values.tolist()

depot_coord = [depot_loc[0][0], depot_loc[0][1]]
def color_depot(coords, default_color):
    if coords == depot_coord:
        return "red"
    else:
        return default_color

def icon_depot(coords, default_icon):
    if coords == depot_coord:
        return "home"
    else:
        return default_icon

def layer_control(coords, name, control=True, show=True):
    if coords == depot_coord:
        fg = folium.FeatureGroup(name, control==False, show)
    else:
        fg = folium.FeatureGroup(name, control==control, show)
    return fg


In [32]:
m = folium.Map(location=(data_dc['lat_customers'].mean(), data_dc['lon_customers'].mean()), zoom_start=8)
fg_icon = folium.FeatureGroup(name='Icon Collection', show=True).add_to(m)
fg_line = folium.FeatureGroup(name='Route', show=False).add_to(m)
fg_line_opt = folium.FeatureGroup(name='Route Optimization', show=False).add_to(m)
marker_cluster_icon = MarkerCluster().add_to(fg_icon)
marker_cluster_line = MarkerCluster().add_to(fg_line)
marker_cluster_opt = MarkerCluster().add_to(fg_line_opt)
folium.map.LayerControl().add_to(m)

# marker icon
for _,row in data_dc.iterrows():
    name = row['customers']
    lat = row['lat_customers']
    lon = row['lon_customers']
    coords = [lat,lon]

    reversed_coords = list(reversed(coords))
    popup = "<strong>{0}</strong><br>Lat: {1:.3f}<br>Long: {2:.3f}".format(name, lat, lon)
    icon = folium.Icon(
        color=color_depot(reversed_coords,"blue"),
        icon_color='white',
        icon=icon_depot(reversed_coords,"info"),  # fetches font-awesome.io symbols
        prefix='fa')
    folium.Marker(coords, icon=icon, popup=popup).add_to(fg_icon)

# marker icon FOR DEPOT
dc_lat = float(data_dc.iloc[0]['DC_latitude'])
dc_lon = float(data_dc.iloc[0]['DC_longitude'])
dc_lon_lat = [dc_lon, dc_lat]

popup = "<strong>{0}</strong><br>Lat: {1:.3f}<br>Long: {2:.3f}".format(name, lat, lon)
icon = folium.Icon(
    color=color_depot(dc_lon_lat,"blue"),
    icon_color='white',
    icon=icon_depot(dc_lon_lat,"info"),  # fetches font-awesome.io symbols
    prefix='fa')
folium.Marker([dc_lat,dc_lon], icon=icon, popup=popup).add_to(fg_icon)

# get random route using direction API Open Route Service
line_colors = ['green', 'orange', 'blue', 'yellow']

decode_geometries = []
# Loop untuk mendekode setiap geometry dari alternatif rute (enumerate untuk bentuk list)
for i, responses in enumerate(random_route):
    color = line_colors[i % len(line_colors)]
    routes = responses['routes']
    coords = responses['metadata']['query']['coordinates']
    # preference = responses['preference']

    for j, route in enumerate(routes):
        geometry = route['geometry']
        distance = route['summary']['distance']
        duration = route['summary']['duration']
        decoded = decode_polyline(geometry)['coordinates']  # Decode geometry
        decode_geometries.append(decoded)  # Simpan ke daftar
        for i, coord in enumerate(coords):
            line = folium.PolyLine(
                locations=[list(reversed(coord)) for coord in decoded],
                color=color,
                weight=5,
                opacity=0.5,
                popup=f"Distance:{distance}{units} \n Duration:{duration / 60:.2f}min"
            )
            marker_cluster_line.add_child(line)

# Memuat file JSON dan mengonversinya menjadi list
color_codes_json_path = 'color_codes.json'

with open(color_codes_json_path, 'r') as file:
    color_codes = json.load(file)

line_colors_opt = color_codes

# get optimize route using direction API Open Route Service based on sequence coordinate OR-TOOLS
for i, value in enumerate(result_route_optimal):
    for vehicle_id in vehicle_id_list:
        color = line_colors_opt[i % len(line_colors_opt)]
        # Periksa jika vehicle_id ada di value
        if vehicle_id in value:
            routes = value[vehicle_id]['routes']
            coords = value[vehicle_id]['metadata']['query']['coordinates']

            for k, route in enumerate(routes):
                geometry = route['geometry']
                distance = route['summary']['distance']
                duration = route['summary']['duration']
                decoded = decode_polyline(geometry)['coordinates']
                for i, coord in enumerate(coords):
                    line_opt = folium.PolyLine(
                        locations=[list(reversed(coord)) for coord in decoded],
                        color=color,
                        weight=2,
                        opacity=0.7,
                        popup=f"Distance:{distance}{units} \n Duration:{duration / 60:.2f}min",
                    )
                    marker_cluster_opt.add_child(line_opt)
m.save(file_map)